In [ ]:
%matplotlib inline

Solving quadratic problems basic
================================

::: {.seealso}
See
`sphx_glr_auto_examples_solvers_ex_quad_problems_advanced.py`{.interpreted-text
role="ref"} for an advanced example how to solve quadratic problems. See
`sphx_glr_auto_examples_solvers_ex_linear_problems_basic.py`{.interpreted-text
role="ref"} for an introduction to solving linear problems. See
`sphx_glr_auto_examples_solvers_ex_linear_problems_advanced.py`{.interpreted-text
role="ref"} for an advanced example how to solve linear problems.
:::


This notebook elaborates on how to quadratic problems, e.g. the
`moscot.problems.time.LineageProblem`{.interpreted-text role="class"},
the
`moscot.problems.spatio_temporal.SpatioTemporalProblem`{.interpreted-text
role="class"}, the
`moscot.problems.space.MappingProblem`{.interpreted-text role="class"},
the `moscot.problems.time.AlignmentProblem`{.interpreted-text
role="class"}, the `moscot.problems.generic.GWProblem`{.interpreted-text
role="class"}, and the the
`moscot.problems.generic.FGWProblem`{.interpreted-text role="class"}.


In [ ]:
from moscot.datasets import simulate_data
from moscot.problems.generic import GWProblem, FGWProblem
import scanpy as sc

import numpy as np

adata = simulate_data(n_distributions=2, key="batch", quad_term="spatial")
sc.pp.pca(adata)
print(adata)

Basic parameters
================

There are some parameters in quadratic problems which play the same role
as in linear problems. Hence, we refer to TODO for the role of
[epsilon]{.title-ref}, [tau\_a]{.title-ref}, and [tau\_b]{.title-ref}.
In fused quadratic problems (also referred to as Fused
Gromov-Wasserstein) there is an additional parameter [alpha]{.title-ref}
defining the convex combination between the quadratic and the linear
term. Setting [alpha=1]{.title-ref} only considers the quadratic term,
while [alpha -\> 0]{.title-ref} only considers the linear term. While
choosing [alpha=0]{.title-ref} is possible in fused quadratic problems,
and corresponds to the pure quadratic problem, [alpha=0]{.title-ref} is
not possible, and hence linear problems must be chosen.


In [ ]:
gwp = GWProblem(adata)
gwp = gwp.prepare(key="batch", GW_x={"attr": "obsm", "key": "spatial"}, GW_y={"attr": "obsm", "key": "spatial"})
gwp = gwp.solve(alpha=0, epsilon=1e-1)

fgwp = FGWProblem(adata)
fgwp = fgwp.prepare(
    key="batch", GW_x={"attr": "obsm", "key": "spatial"}, GW_y={"attr": "obsm", "key": "spatial"}, joint_attr="X_pca"
)
fgwp = fgwp.solve(epsilon=1e-1)

max_difference = np.max(np.abs(gwp["0", "1"].solution.transport_matrix - fgwp["0", "1"].solution.transport_matrix))
print(f"{max_difference:.6f}")

Low-rank solutions
==================

Whenever the dataset is very large, the computational complexity can be
reduced by setting [rank]{.title-ref} to a positive integer
(`scetbon:21a`{.interpreted-text role="cite"}). In this case,
[epsilon]{.title-ref} can also be set to 0, while only the balanced case
([tau\_a = tau\_b = 1]{.title-ref}) is supported. Moreover, the data has
to be provided as point clouds, i.e. no precomputed cost matrix can be
passed.


In [ ]:
gwp = gwp.solve(epsilon=1e-2, rank=3)

Scaling the cost
================

[scale\_cost]{.title-ref} works the same way as for linear problems.
Note that all cost terms will be scaled by the same argument.
